# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../data/SampleHDS.jso…",1.2897e19,"""12909024528636470621""","""9377565583492680131""","""6233164043418656606""","""18169720978691205270""",false
"""../../../../data/SampleHDS.jso…",6.0128e18,"""17707244791594969277""","""2352487729845830649""","""6233164043418656606""","""1351408004757087997""",true
"""../../../../data/SampleHDS.jso…",1.5699e19,"""7256350630096402728""","""15348478728963963985""","""432614189691239930""","""18169720978691205270""",false
"""../../../../data/SampleHDS.jso…",3.9679e18,"""6105769019553157524""","""2352487729845830649""","""3356965197068441333""","""18169720978691205270""",false
"""../../../../data/SampleHDS.jso…",5.2074e18,"""10551454634577828652""","""9377565583492680131""","""3356965197068441333""","""16338586326056824528""",true
"""../../../../data/SampleHDS.jso…",5.9830e18,"""6151647156164913094""","""15348478728963963985""","""11012787168820128525""",null,false
"""../../../../data/SampleHDS.jso…",1.7473e19,"""10380293259118513862""","""15348478728963963985""","""3356965197068441333""","""16338586326056824528""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../data/SampleHDS.jso…",1.8056e19,"""5269484998001570564""","""8581273182524607816""","""FirstMortgage30yr""","""6296857352686402207""","""Rejected"""
"""../../../../data/SampleHDS.jso…",3.3270e18,"""7238239724914238892""","""1879821103390867871""","""FirstMortgage30yr""","""15329476834504036002""","""Accepted"""
"""../../../../data/SampleHDS.jso…",1.6442e19,"""1946741621050522680""","""7515386579679880976""","""MoneyMarketSavingsAccount""","""6296857352686402207""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.9213e17,"""1733660499531062518""","""1879821103390867871""","""BasicChecking""","""6296857352686402207""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.4978e19,"""12813509641999681462""","""8581273182524607816""","""BasicChecking""","""12450208953703215581""","""Accepted"""
"""../../../../data/SampleHDS.jso…",6.8356e18,"""5565571780730225539""","""7515386579679880976""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…",1.7814e19,"""4984613486906898080""","""7515386579679880976""","""BasicChecking""","""12450208953703215581""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../data/SampleHDS.jso…",1.0516e19,"""5021772882474508070""","""229428890612570915""","""FirstMortgage30yr""","""9225554178279845854""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.3584e19,"""10056157728048751510""","""2101692440506464343""","""FirstMortgage30yr""","""13581516073519361656""","""Accepted"""
"""../../../../data/SampleHDS.jso…",4.3045e18,"""6140892064341672210""","""1235812855877742332""","""MoneyMarketSavingsAccount""","""9225554178279845854""","""Rejected"""
"""../../../../data/SampleHDS.jso…",5.8363e18,"""7950818606750344316""","""2101692440506464343""","""BasicChecking""","""9225554178279845854""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.6695e19,"""11060479061296157359""","""229428890612570915""","""BasicChecking""","""17271784727653799327""","""Accepted"""
"""../../../../data/SampleHDS.jso…",1.3767e19,"""11465338383022068053""","""1235812855877742332""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…",9.8057e18,"""2859093072474413674""","""1235812855877742332""","""BasicChecking""","""17271784727653799327""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.5236e19,"""16597522446791390221""","""13287517305332894479""","""FirstMortgage30yr""","""4918662865459119115""","""Rejected"""
2.7421e18,"""12154421659178929289""","""13319023010309524622""","""FirstMortgage30yr""","""15384039588065283733""","""Accepted"""
4.8185e18,"""500843542021673737""","""13060139482065217693""","""MoneyMarketSavingsAccount""","""4918662865459119115""","""Rejected"""
4.1557e18,"""13480559052404381479""","""13319023010309524622""","""BasicChecking""","""4918662865459119115""","""Rejected"""
6.7207e18,"""2455861621109907832""","""13287517305332894479""","""BasicChecking""","""3334912967268865292""","""Accepted"""
1.6426e19,"""13411320500474434145""","""13060139482065217693""","""UPlusFinPersonal""",null,"""Rejected"""
1.4140e19,"""17249902821003769084""","""13060139482065217693""","""BasicChecking""","""3334912967268865292""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'